# Link Prediction

In this notebook, we demonstrate how to use GraphLab Create to construct a simple [Link Prediction](http://personal.stevens.edu/~jbao/BIA658A/Session10/liben-nowell.pdf) classifier.
A link prediction classifier is a classifier that can predict the probability of the existence (or non-existence) of a link in a social network (SN). Namely, given a social network, such as Facebook, a link prediction classifier can help to determine if a link between two users exist. Our construction is general and not limited to SN. For example, in a phone call network our classifier can predict whether two subscribers will call each other. 

There are many diverse methods to construct a link prediction classifier, such as using the SN topology, using the SN users' personal details or using posts and other published online content. 

The main goal of this notebook is to demonstrate how to use GraphLab's SFrame and SGraph to extract various SN topological features. These topological features can later be utilized to perform various prediction tasks, such as 
[link prediction](http://dl.acm.org/citation.cfm?id=2542192), and [predicting students' exam scores](http://www.academia.edu/1123106/Predicting_Student_Exams_Scores_by_Analyzing_Social_Network_Data). 

This notebook is organized as follows: First, we download a link prediction training dataset and constract a graph from it. Next, we will illustrate how GraphLab's SFrame and SGraph objects can be utilized to extract various SN topological features for each link. Finally, we will demonstrate how to use those features inside a link prediction classifier.

## Downloading the Dataset 

In this notebook, we will use dataset which was published by [BGU Social Network Research Group](http://proj.ise.bgu.ac.il/sns/datasets.html). 
We will download the [Google+ dataset](http://proj.ise.bgu.ac.il/sns/googlep.html) which consists of around 3 million links, and generate a SGraph object containing the social links.

In [1]:
import graphlab as gl
import networkx as nx
gl.product_key.set_product_key('DEAD-3D52-156B-178F-F077-EB67-480D-23DA')


# Loading the links dataset into a SFrame object
# sf_links = gl.SFrame.read_csv("https://static.turi.com/datasets/bgu_directed_network_googleplus/g_plus_pos_and_neg_links.csv.gz")

sf_links = gl.SFrame.read_csv("C:/Users/Modarres/Desktop/sample.txt") # This is the subgraph file, use the 2mb file for now


# Let's view the data
print sf_links.head(3)


# Creating SGraph object from the SFrame object
g = gl.SGraph().add_edges(sf_links, src_field='src', dst_field='dst')






This non-commercial license of GraphLab Create for academic use is assigned to smodarres@student.unimelb.edu.au and will expire on August 23, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Modarres\AppData\Local\Temp\graphlab_server_1535362062.log.0


Finished parsing file C:\Users\Modarres\Desktop\sample.txt

Parsing completed. Parsed 100 lines in 0.05773 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\Modarres\Desktop\sample.txt

Parsing completed. Parsed 114132 lines in 0.057042 secs.

+---------+---------+-------+
|   src   |   dst   | class |
+---------+---------+-------+
| 1146906 | 4122183 |   1   |
| 2448032 | 3401703 |   0   |
| 2290639 |  996371 |   0   |
+---------+---------+-------+
[3 rows x 3 columns]



In [2]:
# added code
sf_links_entire = gl.SFrame.read_csv("C:/Users/Modarres/Desktop/data.txt") # this is the all the train data in edge format. add src,dst as first line
g_entire = gl.SGraph().add_edges(sf_links_entire, src_field='src', dst_field='dst')
print sf_links_entire.head(10)
print g_entire.get_edges(dst_ids=[4066935])


# this is the test file with test edges that we need to produce probabilities for, only keep the first 5 lines because currently this is very slow
sf_links_test = gl.SFrame.read_csv("C:/Users/Modarres/Desktop/test_data.txt", delimiter="\t") 

print sf_links_test.head(5)


g_test = gl.SGraph().add_edges(sf_links_test, src_field='Source', dst_field='Sink')


Finished parsing file C:\Users\Modarres\Desktop\data.txt

Parsing completed. Parsed 100 lines in 0.909735 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 3246017 lines. Lines per second: 2.34038e+006

Finished parsing file C:\Users\Modarres\Desktop\data.txt

...

Parsing completed. Parsed 7 lines in 0.018999 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\Modarres\Desktop\test_data.txt

Parsing completed. Parsed 7 lines in 0.025331 secs.

+----+---------+---------+
| Id |  Source |   Sink  |
+----+---------+---------+
| 1  | 2184483 | 1300190 |
| 2  | 3151356 | 1452193 |
| 3  | 1579396 |  193159 |
| 4  | 1406432 | 2481036 |
| 5  | 2389638 |  593017 |
+----+---------+---------+
[5 rows x 3 columns]



Let use the SGraph's summary function to get details on the loaded SGraph object

In [3]:
g.summary()

{'num_edges': 114132L, 'num_vertices': 31472L}

Our directed graph has around 3 million edges (referred to as links) and over 200,000 vertices (referred to as users). 
Those links are roughtly divided into two. Positive links (labeled 1), and Negative links (labeled 0). The positive links are observed links in the social network, while negative links were added at random so our binary classifier could learn from the negative examples as well. Note, that if you like to use your own dataset, you will need to add negative (unobserved) edges at random for our construction to work.

## Topological Feature Extraction

Let's create an SFrame obejct with all user ids.

In [4]:
users_sf = g.get_vertices()
users_sf.rename({"__id": "id"}) 
users_sf.head(5)



id
115
2060
3758
4084
6993


In [5]:
#added code
users_sf_test = g_test.get_vertices()
users_sf_test.rename({"__id": "id"}) 
users_sf_test.head(5)

id
1237964
1452193
2389638
3151356
212805


The users sframe can be used for calculating several topological feature for each link in the dataset. 
We will start by calculating various simple topological features, such as the each user's in-degree (i.e. the user's number of followers) and user's out-degree (i.e. number of users the user is followling).  

To calculate each vertex toplogical features, we will first create a SFrame object that contains each user's in-friends (i.e. the users in the network that follows the user), and the user's out-friends (the users in the network which the user follows).

In [6]:
# Calculating each vertices in and out degree
out_friends_sf = sf_links.groupby("src", {"out_friends": gl.aggregate.CONCAT("dst")})
out_friends_sf.rename({"src": "id"})
in_friends_sf = sf_links.groupby("dst", {"in_friends": gl.aggregate.CONCAT("src")})
in_friends_sf.rename({"dst": "id"})




id,in_friends
2759692,"[1812114, 1085187]"
127950,"[3855254, 1729215,2593244, 1349133, 651 ..."
910362,"[2021460, 2023163]"
2067382,[241370]
3561229,"[3068613, 2651652,3166930] ..."
4284688,[1324992]
3742828,"[3854370, 2256492]"
2685078,"[1331326, 1508784,1596155] ..."
1472708,"[378745, 2795881,4139465, 4671336, ..."
460603,[2468519]


In [7]:
#added code
# This section should be straightforward find out why it's so slow!? it takes so much time to find in and out friends for only 6 vertices!


def in_friends(u):
    return g_entire.get_edges(dst_ids=[u]).select_column("__src_id")
#     return g_entire.get_edges(dst_ids=[u])

def out_friends(u):
    return g_entire.get_edges(src_ids=[u]).select_column("__dst_id")


#users_sf_test.add_column("in_friends")
#users_sf_test.add_column("out_friends")

#vertices_infriends = users_sf_test.select_column("id")

users_sf_test['in_friends'] = users_sf_test[['id']].apply(lambda r: in_friends(r['id']))
users_sf_test['out_friends'] = users_sf_test[['id']].apply(lambda r: out_friends(r['id']))

# users_sf_test = users_sf_test.fillna('in_friends',[])
# users_sf_test = users_sf_test.fillna('out_friends',[])

print users_sf_test



+---------+-------------------------------+-------------------------------+
|    id   |           in_friends          |          out_friends          |
+---------+-------------------------------+-------------------------------+
| 1237964 | [448823.0, 4157152.0, 3317... | [2332893.0, 448823.0, 1139... |
| 1452193 | [2501645.0, 3660919.0, 884... |               []              |
| 2389638 | [2880517.0, 3553485.0, 212... | [201039.0, 2402120.0, 2819... |
| 3151356 | [2120801.0, 1357350.0, 408... | [4052824.0, 3317934.0, 184... |
|  212805 | [68695.0, 884795.0, 279170... | [620720.0, 2576942.0, 2728... |
|  593017 | [974241.0, 3694560.0, 1181... | [1328064.0, 595768.0, 2704... |
| 1300190 | [4651416.0, 3227840.0, 265... |               []              |
| 2184483 | [3770256.0, 2173130.0, 465... | [3209755.0, 1687276.0, 432... |
|  228206 | [1869658.0, 4738810.0, 763... | [76360.0, 1869658.0, 47388... |
| 1406432 | [1687276.0, 972843.0, 2956... | [4088654.0, 1351603.0, 200... |
+---------+-

Using the SFrame [join](https://turi.com/products/create/docs/generated/graphlab.SFrame.join.html#graphlab.SFrame.join) operation, we create a single SFrame which consists of each user's in and out friends.

In [8]:
users_sf = users_sf.join(in_friends_sf, on="id", how="outer")
users_sf = users_sf.join(out_friends_sf, on="id", how="outer")

# we replace missing values with empty lists
users_sf = users_sf.fillna('in_friends',[])
users_sf = users_sf.fillna('out_friends',[])
users_sf.head(10)

id,in_friends,out_friends
2759692,"[1812114, 1085187]","[251505, 990399]"
127950,"[3855254, 1729215,2593244, 1349133, 651 ...",[1123570]
910362,"[2021460, 2023163]",[]
2067382,[241370],[]
3561229,"[3068613, 2651652,3166930] ...","[259166, 4108731]"
4284688,[1324992],[]
3742828,"[3854370, 2256492]",[]
2685078,"[1331326, 1508784,1596155] ...",[]
1472708,"[378745, 2795881,4139465, 4671336, ...","[3637026, 438950,2085431, 1668027, 848 ..."
460603,[2468519],"[3694826, 396073]"


Using the created SFrame with each user in-friends and out-friends, we calculate several simple topological features for each user,  such as each user's in-degree (i.e. number of followers) and each user's out-degree (i.e. number of users the user is followling). 

In [9]:
#out_degree - number of users each vertex is following
users_sf['out_degree'] = users_sf["out_friends"].apply(lambda l: len(l) )

#in_degree - number of users following each vertex
users_sf['in_degree'] = users_sf["in_friends"].apply(lambda l: len(l) )

#all_degree - number of uniuqe users that following or are followed by each user
users_sf['all_friends'] = users_sf[['in_friends', 'out_friends']].apply(lambda r: list(set(r['in_friends']) | set(r['out_friends'])))
users_sf['all_degree'] = users_sf["all_friends"].apply(lambda l: len(l) )

#bi_degree - number of uniuqe users that are both following and followed by each user
users_sf['bi_friends'] = users_sf[['in_friends', 'out_friends']].apply(lambda r: list(set(r['in_friends']) & set(r['out_friends'])))
users_sf['bi_degree'] = users_sf["bi_friends"].apply(lambda l: len(l) )

users_sf.head(10)




id,in_friends,out_friends,out_degree,in_degree,all_friends
2759692,"[1812114, 1085187]","[251505, 990399]",2,2,"[251505.0, 1812114.0,1085187.0, 990399.0] ..."
127950,"[3855254, 1729215,2593244, 1349133, 651 ...",[1123570],1,17,"[651778.0, 1349133.0,4497240.0, 3855254.0, ..."
910362,"[2021460, 2023163]",[],0,2,"[2023163.0, 2021460.0]"
2067382,[241370],[],0,1,[241370.0]
3561229,"[3068613, 2651652,3166930] ...","[259166, 4108731]",2,3,"[3166930.0, 4108731.0,2651652.0, 3068613.0, ..."
4284688,[1324992],[],0,1,[1324992.0]
3742828,"[3854370, 2256492]",[],0,2,"[3854370.0, 2256492.0]"
2685078,"[1331326, 1508784,1596155] ...",[],0,3,"[1508784.0, 1596155.0,1331326.0] ..."
1472708,"[378745, 2795881,4139465, 4671336, ...","[3637026, 438950,2085431, 1668027, 848 ...",7,21,"[901184.0, 4139465.0,1702417.0, 3353876.0, ..."
460603,[2468519],"[3694826, 396073]",2,1,"[396073.0, 3694826.0,2468519.0] ..."


In [10]:
# added code
# why so slow?!

#out_degree - number of users each vertex is following
users_sf_test['out_degree'] = users_sf_test["out_friends"].apply(lambda l: len(l) )

#in_degree - number of users following each vertex
users_sf_test['in_degree'] = users_sf_test["in_friends"].apply(lambda l: len(l) )

#all_degree - number of uniuqe users that following or are followed by each user
users_sf_test['all_friends'] = users_sf_test[['in_friends', 'out_friends']].apply(lambda r: list(set(r['in_friends']) | set(r['out_friends'])))
users_sf_test['all_degree'] = users_sf_test["all_friends"].apply(lambda l: len(l) )

#bi_degree - number of uniuqe users that are both following and followed by each user
users_sf_test['bi_friends'] = users_sf_test[['in_friends', 'out_friends']].apply(lambda r: list(set(r['in_friends']) & set(r['out_friends'])))
users_sf_test['bi_degree'] = users_sf_test["bi_friends"].apply(lambda l: len(l) )

users_sf_test.head(10)

id,in_friends,out_friends,out_degree,in_degree,all_friends
1237964,"[448823.0, 4157152.0,3317847.0, 1601895.0] ...","[2332893.0, 448823.0,1139433.0, 1413055.0, ...",15,4,"[893888.0, 959493.0,2754231.0, 4326349.0, ..."
1452193,"[2501645.0, 3660919.0,884795.0, 4558216.0, ...",[],0,289,"[3671045.0, 2501645.0,905235.0, 4864020.0, ..."
2389638,"[2880517.0, 3553485.0,2120801.0, 4003368.0, ...","[201039.0, 2402120.0,2819498.0, 4553431.0, ...",267,30,"[4332544.0, 4475906.0,500739.0, 2880517.0, ..."
3151356,"[2120801.0, 1357350.0,4088506.0, 1759823.0, ...","[4052824.0, 3317934.0,1848747.0, 2482853.0, ...",340,39,"[2966358.0, 2587654.0,2270215.0, 957452.0, ..."
212805,"[68695.0, 884795.0,2791702.0, 1328244.0, ...","[620720.0, 2576942.0,2728232.0, 1439892.0, ...",297,895,"[3971074.0, 431384.0,2209796.0, 4821000.0, ..."
593017,"[974241.0, 3694560.0,1181282.0, 2032058.0, ...","[1328064.0, 595768.0,2704984.0, 2877896.0, ...",58,165,"[99328.0, 1395712.0,541698.0, 4055513.0, ..."
1300190,"[4651416.0, 3227840.0,2659783.0] ...",[],0,3,"[4651416.0, 3227840.0,2659783.0] ..."
2184483,"[3770256.0, 2173130.0,4653515.0, 1729215.0, ...","[3209755.0, 1687276.0,4325955.0, 3057131.0, ...",83,102,"[1340930.0, 2651652.0,2146309.0, 1706508.0, ..."
228206,"[1869658.0, 4738810.0,76360.0, 986629.0, ...","[76360.0, 1869658.0,4738810.0, 2308522.0, ...",9,14,"[110976.0, 2298370.0,212805.0, 4003975.0, ..."
1406432,"[1687276.0, 972843.0,2956481.0, 3504137.0, ...","[4088654.0, 1351603.0,2002799.0, 739382.0, ...",84,16,"[2304782.0, 669843.0,1984005.0, 4013831.0, ..."


Now, we  have several degree feautres for each user. Lets utilize these users' features and create features for each link in our positive and negative links dataset.
Namely, for each link in the data conists of two users <i>u</i> and <i>v</i>, we will create SFrame with each user's degree features.

Note: runining can take several minutes.

In [11]:

sf_links = sf_links.join(users_sf, on={"src": "id"}, how="right")
sf_links.rename({"in_friends": "src_in_friends", "out_friends": "src_out_friends",
           "all_friends": "src_all_friends", "all_degree": "src_all_degree",
           "bi_friends": "src_bi_friends", "bi_degree": "src_bi_degree",
           "in_degree": "src_in_degree", "out_degree": "src_out_degree"
           })

sf_links = sf_links.join(users_sf, on={"dst": "id"}, how="right")
sf_links.rename({"in_friends": "dst_in_friends", "out_friends": "dst_out_friends",
           "all_friends": "dst_all_friends", "all_degree": "dst_all_degree",
           "bi_friends": "dst_bi_friends", "bi_degree": "dst_bi_degree",
           "in_degree": "dst_in_degree", "out_degree": "dst_out_degree"})

sf_links.head(10)


src,dst,class,src_in_friends,src_out_friends,src_out_degree,src_in_degree
1146906,4122183,1,"[3353876, 1916723,4030043, 2916456, ...","[4122183, 4751012,3844222, 3953964, ...",601,211
2448032,3401703,0,[3000657],[3401703],1,1
2290639,996371,0,"[4488717, 3457327,3806350, 3144384, ...","[996371, 302860, 1799182,2227954, 2940560, ...",108,62
1146906,4751012,1,"[3353876, 1916723,4030043, 2916456, ...","[4122183, 4751012,3844222, 3953964, ...",601,211
1324992,2533043,1,"[3609326, 2272405,3143728, 2300708, ...","[2533043, 2502555,2719963, 3240066, ...",290,105
651778,1363531,1,"[4603631, 3539009,3314478, 2842633, ...","[1363531, 1438067,1178165, 3613981, ...",396,153
4217935,4307379,1,"[3961108, 3402168,1719385, 4251081, ...","[4307379, 1602251,2435840, 3253984, ...",584,227
1988622,326813,1,"[1702417, 991226,3355142, 1574731, ...","[326813, 163021, 3678455,505037, 3170235, 577732, ...",371,147
2385853,2135370,0,"[977900, 2499714,1122854, 3459341, 990 ...","[2135370, 4200163,2940560, 996371, 946951, ...",393,220
1146906,3844222,1,"[3353876, 1916723,4030043, 2916456, ...","[4122183, 4751012,3844222, 3953964, ...",601,211


In [12]:
#added code
sf_links_test = sf_links_test.join(users_sf_test, on={"Source": "id"}, how="inner")
sf_links_test.rename({"in_friends": "src_in_friends", "out_friends": "src_out_friends",
           "all_friends": "src_all_friends", "all_degree": "src_all_degree",
           "bi_friends": "src_bi_friends", "bi_degree": "src_bi_degree",
           "in_degree": "src_in_degree", "out_degree": "src_out_degree"
           })

sf_links_test = sf_links_test.join(users_sf_test, on={"Sink": "id"}, how="inner")
sf_links_test.rename({"in_friends": "dst_in_friends", "out_friends": "dst_out_friends",
           "all_friends": "dst_all_friends", "all_degree": "dst_all_degree",
           "bi_friends": "dst_bi_friends", "bi_degree": "dst_bi_degree",
           "in_degree": "dst_in_degree", "out_degree": "dst_out_degree"})

sf_links_test.head(10)


Columns:
	Id	int
	Source	int
	Sink	int
	src_in_friends	array
	src_out_friends	array
	src_out_degree	int
	src_in_degree	int
	src_all_friends	array
	src_all_degree	int
	src_bi_friends	array
	src_bi_degree	int
	dst_in_friends	array
	dst_out_friends	array
	dst_out_degree	int
	dst_in_degree	int
	dst_all_friends	array
	dst_all_degree	int
	dst_bi_friends	array
	dst_bi_degree	int

Rows: 7

Data:
+----+---------+---------+-------------------------------+
| Id |  Source |   Sink  |         src_in_friends        |
+----+---------+---------+-------------------------------+
| 2  | 3151356 | 1452193 | [2120801.0, 1357350.0, 408... |
| 6  |  228206 |  212805 | [1869658.0, 4738810.0, 763... |
| 5  | 2389638 |  593017 | [2880517.0, 3553485.0, 212... |
| 1  | 2184483 | 1300190 | [3770256.0, 2173130.0, 465... |
| 3  | 1579396 |  193159 | [2984603.0, 2120801.0, 274... |
| 7  | 1237964 |  879115 | [448823.0, 4157152.0, 3317... |
| 4  | 1406432 | 2481036 | [1687276.0, 972843.0, 2956... |
+----+---------+---------+-------------------------------+
+-------------------------------+----------------+---------------+
|        src_out_friends        | src_out_degree | src_in_degree |
+-------------------------------+----------------+---------------+
| [4052824.0, 3317934.0, 184... |      340       |       39      |
| [76360.0, 1869658.0, 47388... |       9        |       14      |
| [201039.0, 2402120.0, 2819... |      267       |       30      |
| [3209755.0, 1687276.0, 432... |       83       |      102      |
| [1519588.0, 916145.0, 4352... |      208       |       14      |
| [2332893.0, 448823.0, 1139... |       15       |       4       |
| [4088654.0, 1351603.0, 200... |       84       |       16      |
+-------------------------------+----------------+---------------+
+-------------------------------+----------------+
|        src_all_friends        | src_all_degree |
+-------------------------------+----------------+
| [2966358.0, 2587654.0, 227... |      354       |
| [110976.0, 2298370.0, 2128... |       16       |
| [4332544.0, 4475906.0, 500... |      284       |
| [1340930.0, 2651652.0, 214... |      145       |
| [2556418.0, 3422215.0, 202... |      209       |
| [893888.0, 959493.0, 27542... |       16       |
| [2304782.0, 669843.0, 1984... |       86       |
+-------------------------------+----------------+
+-------------------------------+---------------+-------------------------------+
|         src_bi_friends        | src_bi_degree |         dst_in_friends        |
+-------------------------------+---------------+-------------------------------+
| [4384662.0, 1357350.0, 351... |       25      | [2501645.0, 3660919.0, 884... |
| [4003975.0, 76360.0, 31038... |       7       | [68695.0, 884795.0, 279170... |
| [20388.0, 2989221.0, 23142... |       13      | [974241.0, 3694560.0, 1181... |
| [1218944.0, 3123328.0, 265... |       40      | [4651416.0, 3227840.0, 265... |
| [2120801.0, 20388.0, 20254... |       12      |     [3278181.0, 4518416.0]    |
| [448823.0, 1601895.0, 3317... |       3       | [2424452.0, 1133307.0, 175... |
| [2152768.0, 2956481.0, 972... |       14      | [700731.0, 317366.0, 22367... |
+-------------------------------+---------------+-------------------------------+
+-------------------------------+----------------+---------------+
|        dst_out_friends        | dst_out_degree | dst_in_degree |
+-------------------------------+----------------+---------------+
|               []              |       0        |      289      |
| [620720.0, 2576942.0, 2728... |      297       |      895      |
| [1328064.0, 595768.0, 2704... |       58       |      165      |
|               []              |       0        |       3       |
|               []              |       0        |       2       |
|               []              |       0        |      482      |
| [3086178.0, 2385853.0, 198... |       14       |       24      |
+-------------------------------+----------------+---------------+
+----------------------

Beside adding each link's users <i> u </i> and <i> v </i> degree features, to create a decent link prediction classifier, we also need to add features based on the strength of connection between the users.
In this notebook, we will add for each link three simple type of features:
- <i>Common-Friends Features</i> - the number of friends both <i>u</i> and <i>v</i> have in common. 
- <i>Total-Friends Features</i> - the number of friends both <i>u</i> and <i>v</i> have in together.
- <i>Jaccard coefficient</i>- the number of Common-Friends divided by the Number of Total-Friends.

Lets define the each feature type function:


In [13]:
def common_friends(u, v, u_friends, v_friends):
    u_friends = set(u_friends)
    if v in u_friends:
            u_friends.remove(v)

    v_friends = set(v_friends)
    if u in v_friends:
        v_friends.remove(u)
    return len(u_friends & v_friends)

def total_friends(u, v, u_friends, v_friends):
    u_friends = set(u_friends)
    if v in u_friends:
        u_friends.remove(v)

    v_friends = set(v_friends)
    if u in v_friends:
        v_friends.remove(u)

    return len(u_friends | v_friends)

def jacc_coef(u,v, u_friends, v_friends):
    t = total_friends(u,v,u_friends,v_friends)
    if  t == 0:
        return 0
    return common_friends(u,v,u_friends, v_friends)/ float(t)


# we add more features like scc here later

Using these three features type we created 12 new features (4 feature for each feature type) that are based on direction of the friendship between 
<i>u</i> and <i>v</i> and their friends. 

Please note that The formal mathematical defintion of the feature presented throught this section can be found in [Fire et al. 2014](http://dl.acm.org/citation.cfm?id=2542192).

The following code block may take a few minutes.

In [14]:
sf_links['common_friends'] = sf_links[['src','dst', 'src_all_friends', 'dst_all_friends']].apply(lambda r: common_friends(r['src'], r['dst'],r['src_all_friends'], r['dst_all_friends']))
sf_links['common_bi_friends'] = sf_links[['src','dst', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: common_friends(r['src'], r['dst'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links['common_in_friends'] = sf_links[['src','dst', 'src_in_friends', 'dst_in_friends']].apply(lambda r: common_friends(r['src'], r['dst'],r['src_in_friends'], r['dst_in_friends']))
sf_links['common_out_friends'] = sf_links[['src','dst', 'src_out_friends', 'dst_out_friends']].apply(lambda r: common_friends(r['src'], r['dst'],r['src_out_friends'], r['dst_out_friends']))

sf_links['total_friends'] = sf_links[['src','dst', 'src_all_friends', 'dst_all_friends']].apply(lambda r: total_friends(r['src'], r['dst'],r['src_all_friends'], r['dst_all_friends']))
sf_links['total_bi_friends'] = sf_links[['src','dst', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: total_friends(r['src'], r['dst'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links['total_in_friends'] = sf_links[['src','dst', 'src_in_friends', 'dst_in_friends']].apply(lambda r: total_friends(r['src'], r['dst'],r['src_in_friends'], r['dst_in_friends']))
sf_links['total_out_friends'] = sf_links[['src','dst', 'src_out_friends', 'dst_out_friends']].apply(lambda r: total_friends(r['src'], r['dst'],r['src_out_friends'], r['dst_out_friends']))


sf_links['jacc_coef'] = sf_links[['src','dst', 'src_all_friends', 'dst_all_friends']].apply(lambda r: jacc_coef(r['src'], r['dst'],r['src_all_friends'], r['dst_all_friends']))
sf_links['bi_jacc_coef'] = sf_links[['src','dst', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: jacc_coef(r['src'], r['dst'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links['in_jacc_coef'] = sf_links[['src','dst', 'src_in_friends', 'dst_in_friends']].apply(lambda r: jacc_coef(r['src'], r['dst'],r['src_in_friends'], r['dst_in_friends']))
sf_links['out_jacc_coef'] = sf_links[['src','dst', 'src_out_friends', 'dst_out_friends']].apply(lambda r: jacc_coef(r['src'], r['dst'],r['src_out_friends'], r['dst_out_friends']))
sf_links.head(10)


src,dst,class,src_in_friends,src_out_friends,src_out_degree,src_in_degree
1146906,4122183,1,"[3353876, 1916723,4030043, 2916456, ...","[4122183, 4751012,3844222, 3953964, ...",601,211
2448032,3401703,0,[3000657],[3401703],1,1
2290639,996371,0,"[4488717, 3457327,3806350, 3144384, ...","[996371, 302860, 1799182,2227954, 2940560, ...",108,62
1146906,4751012,1,"[3353876, 1916723,4030043, 2916456, ...","[4122183, 4751012,3844222, 3953964, ...",601,211
1324992,2533043,1,"[3609326, 2272405,3143728, 2300708, ...","[2533043, 2502555,2719963, 3240066, ...",290,105
651778,1363531,1,"[4603631, 3539009,3314478, 2842633, ...","[1363531, 1438067,1178165, 3613981, ...",396,153
4217935,4307379,1,"[3961108, 3402168,1719385, 4251081, ...","[4307379, 1602251,2435840, 3253984, ...",584,227
1988622,326813,1,"[1702417, 991226,3355142, 1574731, ...","[326813, 163021, 3678455,505037, 3170235, 577732, ...",371,147
2385853,2135370,0,"[977900, 2499714,1122854, 3459341, 990 ...","[2135370, 4200163,2940560, 996371, 946951, ...",393,220
1146906,3844222,1,"[3353876, 1916723,4030043, 2916456, ...","[4122183, 4751012,3844222, 3953964, ...",601,211


In [15]:
#added code

sf_links_test['common_friends'] = sf_links_test[['Source','Sink', 'src_all_friends', 'dst_all_friends']].apply(lambda r: common_friends(r['Source'], r['Sink'],r['src_all_friends'], r['dst_all_friends']))
sf_links_test['common_bi_friends'] = sf_links_test[['Source','Sink', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: common_friends(r['Source'], r['Sink'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links_test['common_in_friends'] = sf_links_test[['Source','Sink', 'src_in_friends', 'dst_in_friends']].apply(lambda r: common_friends(r['Source'], r['Sink'],r['src_in_friends'], r['dst_in_friends']))
sf_links_test['common_out_friends'] = sf_links_test[['Source','Sink', 'src_out_friends', 'dst_out_friends']].apply(lambda r: common_friends(r['Source'], r['Sink'],r['src_out_friends'], r['dst_out_friends']))

sf_links_test['total_friends'] = sf_links_test[['Source','Sink', 'src_all_friends', 'dst_all_friends']].apply(lambda r: total_friends(r['Source'], r['Sink'],r['src_all_friends'], r['dst_all_friends']))
sf_links_test['total_bi_friends'] = sf_links_test[['Source','Sink', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: total_friends(r['Source'], r['Sink'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links_test['total_in_friends'] = sf_links_test[['Source','Sink', 'src_in_friends', 'dst_in_friends']].apply(lambda r: total_friends(r['Source'], r['Sink'],r['src_in_friends'], r['dst_in_friends']))
sf_links_test['total_out_friends'] = sf_links_test[['Source','Sink', 'src_out_friends', 'dst_out_friends']].apply(lambda r: total_friends(r['Source'], r['Sink'],r['src_out_friends'], r['dst_out_friends']))


sf_links_test['jacc_coef'] = sf_links_test[['Source','Sink', 'src_all_friends', 'dst_all_friends']].apply(lambda r: jacc_coef(r['Source'], r['Sink'],r['src_all_friends'], r['dst_all_friends']))
sf_links_test['bi_jacc_coef'] = sf_links_test[['Source','Sink', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: jacc_coef(r['Source'], r['Sink'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links_test['in_jacc_coef'] = sf_links_test[['Source','Sink', 'src_in_friends', 'dst_in_friends']].apply(lambda r: jacc_coef(r['Source'], r['Sink'],r['src_in_friends'], r['dst_in_friends']))
sf_links_test['out_jacc_coef'] = sf_links_test[['Source','Sink', 'src_out_friends', 'dst_out_friends']].apply(lambda r: jacc_coef(r['Source'], r['Sink'],r['src_out_friends'], r['dst_out_friends']))
sf_links_test.head(10)

Columns:
	Id	int
	Source	int
	Sink	int
	src_in_friends	array
	src_out_friends	array
	src_out_degree	int
	src_in_degree	int
	src_all_friends	array
	src_all_degree	int
	src_bi_friends	array
	src_bi_degree	int
	dst_in_friends	array
	dst_out_friends	array
	dst_out_degree	int
	dst_in_degree	int
	dst_all_friends	array
	dst_all_degree	int
	dst_bi_friends	array
	dst_bi_degree	int
	common_friends	int
	common_bi_friends	int
	common_in_friends	int
	common_out_friends	int
	total_friends	int
	total_bi_friends	int
	total_in_friends	int
	total_out_friends	int
	jacc_coef	float
	bi_jacc_coef	float
	in_jacc_coef	float
	out_jacc_coef	float

Rows: 7

Data:
+----+---------+---------+-------------------------------+
| Id |  Source |   Sink  |         src_in_friends        |
+----+---------+---------+-------------------------------+
| 2  | 3151356 | 1452193 | [2120801.0, 1357350.0, 408... |
| 6  |  228206 |  212805 | [1869658.0, 4738810.0, 763... |
| 5  | 2389638 |  593017 | [2880517.0, 3553485.0, 212... |
| 1  | 2184483 | 1300190 | [3770256.0, 2173130.0, 465... |
| 3  | 1579396 |  193159 | [2984603.0, 2120801.0, 274... |
| 7  | 1237964 |  879115 | [448823.0, 4157152.0, 3317... |
| 4  | 1406432 | 2481036 | [1687276.0, 972843.0, 2956... |
+----+---------+---------+-------------------------------+
+-------------------------------+----------------+---------------+
|        src_out_friends        | src_out_degree | src_in_degree |
+-------------------------------+----------------+---------------+
| [4052824.0, 3317934.0, 184... |      340       |       39      |
| [76360.0, 1869658.0, 47388... |       9        |       14      |
| [201039.0, 2402120.0, 2819... |      267       |       30      |
| [3209755.0, 1687276.0, 432... |       83       |      102      |
| [1519588.0, 916145.0, 4352... |      208       |       14      |
| [2332893.0, 448823.0, 1139... |       15       |       4       |
| [4088654.0, 1351603.0, 200... |       84       |       16      |
+-------------------------------+----------------+---------------+
+-------------------------------+----------------+
|        src_all_friends        | src_all_degree |
+-------------------------------+----------------+
| [2966358.0, 2587654.0, 227... |      354       |
| [110976.0, 2298370.0, 2128... |       16       |
| [4332544.0, 4475906.0, 500... |      284       |
| [1340930.0, 2651652.0, 214... |      145       |
| [2556418.0, 3422215.0, 202... |      209       |
| [893888.0, 959493.0, 27542... |       16       |
| [2304782.0, 669843.0, 1984... |       86       |
+-------------------------------+----------------+
+-------------------------------+---------------+-------------------------------+
|         src_bi_friends        | src_bi_degree |         dst_in_friends        |
+-------------------------------+---------------+-------------------------------+
| [4384662.0, 1357350.0, 351... |       25      | [2501645.0, 3660919.0, 884... |
| [4003975.0, 76360.0, 31038... |       7       | [68695.0, 884795.0, 279170... |
| [20388.0, 2989221.0, 23142... |       13      | [974241.0, 3694560.0, 1181... |
| [1218944.0, 3123328.0, 265... |       40      | [4651416.0, 3227840.0, 265... |
| [2120801.0, 20388.0, 20254... |       12      |     [3278181.0, 4518416.0]    |
| [448823.0, 1601895.0, 3317... |       3       | [2424452.0, 1133307.0, 175... |
| [2152768.0, 2956481.0, 972... |       14      | [700731.0, 317366.0, 22367... |
+-------------------------------+---------------+-------------------------------+
+-------------------------------+----------------+---------------+
|        dst_out_friends        | dst_out_degree | dst_in_degree |
+-------------------------------+----------------+---------------+
|               []              |       0        |      289      |
| [620720.0, 2576942.0, 2728... |      297       |      895      |
| [1328064.0, 595768.0, 2704... |       58       |      165      |
|               []              |       0        |       3       |
|               []              |    

Similar to the above method, we can also extract for each link, such as each user [PageRank](https://turi.com/products/create/docs/generated/graphlab.pagerank.create.html) and [neighborhood subgraph](https://turi.com/products/create/docs/generated/graphlab.SGraph.get_neighborhood.html?highlight=neighborhood#graphlab.SGraph.get_neighborhood) size. We let the reader to try to add these features (and  maybe some additional features) by themselves. 

Let us move to the next section, in which we explain how the extracted links' features can be utilized to create a link prediction classifier.

## Constructing a Link Prediction Classifier

In order to create a link prediction classifier using our constructed links dataset (sf), lets first randomly split our dataset into training that contains 20% of the dataset, and testing datasets that contains 80% of the dataset.

In [16]:
train, test = sf_links.random_split(0.02)

We now can use GraphLab Create's [classfication toolkit](https://turi.com/products/create/docs/graphlab.toolkits.classifier.html#creating-a-classifier) and create and evaluate a link prediction classifier based only on <i>u</i> and <i>v</i> degree features:


In [18]:
degree_features_list = [c for c in train.column_names() if "degree" in c]
print "Degree Features %s" % degree_features_list
cls = gl.classifier.create(train,features=degree_features_list, target="class")
results = cls.evaluate(test)
print results

Degree Features ['src_out_degree', 'src_in_degree', 'src_all_degree', 'src_bi_degree', 'dst_out_degree', 'dst_in_degree', 'dst_all_degree', 'dst_bi_degree']
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, SVMClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 2109

Number of classes           : 2

...

| 3         | 4        | 0.011694     | 0.786629          | 0.796748            |

| 4         | 5        | 0.015839     | 0.787103          | 0.796748            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

{'f1_score': 0.8372059156661597, 'auc': 0.8806094235557368, 'recall': 0.9247390620531885, 'precision': 0.7648110920593626, 'log_loss': 0.5655022979126014, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 100001

Data:
+-----------+-----+-----+-------+-------+
| threshold | fpr | tpr |   p   |   n   |
+-----------+-----+-----+-------+-------+
|    0.0    | 1.0 | 1.0 | 55952 | 55948 |
|   1e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   2e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   3e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   4e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   5e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   6e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   7e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   8e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   9e-05   | 1.0 | 1.0 | 55952 | 55948 |
+-----------+-----+-----+-------+-------+
[100001 rows x 5 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'confu

We get pretty good accuracy of 0.88. Let us add also the link based features and use the Boosted-Trees classifier to create and evaluate a link predicdiction classifier.

In [19]:
link_features_list = ['common_friends', 'common_in_friends', 'common_out_friends', 'common_bi_friends', 'total_friends', 'total_in_friends', 'total_out_friends', 'total_bi_friends',
'jacc_coef', 'bi_jacc_coef', 'in_jacc_coef', 'out_jacc_coef']
cls = gl.classifier.boosted_trees_classifier.create(train,target="class", features=degree_features_list + link_features_list)
results = cls.evaluate(test)
print results

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 2117

Number of classes           : 2

...

| 4         | 0.032752     | 0.866320          | 0.808696            | 0.377030          | 0.447958            |

| 5         | 0.037739     | 0.870099          | 0.808696            | 0.346738          | 0.427582            |

| 6         | 0.040732     | 0.871989          | 0.808696            | 0.328025          | 0.408009            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

{'f1_score': 0.845771390407401, 'auc': 0.9083797623895049, 'recall': 0.9166428367171862, 'precision': 0.7850724792970962, 'log_loss': 0.3615956576696605, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 100001

Data:
+-----------+-----+-----+-------+-------+
| threshold | fpr | tpr |   p   |   n   |
+-----------+-----+-----+-------+-------+
|    0.0    | 1.0 | 1.0 | 55952 | 55948 |
|   1e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   2e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   3e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   4e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   5e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   6e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   7e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   8e-05   | 1.0 | 1.0 | 55952 | 55948 |
|   9e-05   | 1.0 | 1.0 | 55952 | 55948 |
+-----------+-----+-----+-------+-------+
[100001 rows x 5 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'confus

In [20]:
print 'probabilites are as follows'
arr = cls.predict(sf_links_test, output_type="probability")
print arr

probabilites are as follows
[0.7222242951393127, 0.024188660085201263, 0.5026636719703674, 0.24718905985355377, 0.7777318358421326, 0.024188660085201263, 0.5399314165115356]


Using the additional link features, we got considerbly better accuracy of around 0.94. We can try to further improve the accuracy by adding additional features or by increasing the size of the training dataset.

